In [2]:
import camelot
import pandas as pd
import numpy as np
import tabula
import re

In [3]:
x=camelot.read_pdf("file:///C:/Users/hp/OneDrive/Documents/BTECH21-Grade-sheet.pdf",pages='all',flavor='stream',edge_tol=1,columns=['2']*8)

In [4]:
for i in x:
    print(i.df)

   0                                                  1
0     ----------------------------------------------...
1     University College of Engineering Vizianagaram...
2     JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY KAKI...
3     HTNO:16VV1A1203 -- II B.Tech I Sem [R17] Regul...
4                  NO\nSUBCODE\nSUBNAME\nGRADE\nCREDITS
5     1\nR1721011\nPROBABILITY & STATISTICS\nF\n0 ( ...
6     2\nR1721051\nMATHEMATICAL FOUNDATIONS OF COMPU...
7     3\nR1721052\nDATA STRUCTURES THOROUGH C++\nF\n...
8      4\nR1721121\nDIGITAL LOGIC DESIGN\nF\n0 ( Fail )
9        5\nR1721122\nPYTHON PROGRAMMING\nD\n3 ( Pass )
10    6\nR1721123\nFORMAL LANGUAGES & AUTOMATA THEOR...
11    7\nR1721124\nDATA STRUCTURES THROUGH C++ LABOR...
12    8\nR1721125\nPYTHON PROGRAMMING LABORATORY\nB\...
13                                             TOTAL\n7
14    ----------------------------------------------...
15    University College of Engineering Vizianagaram...
16    JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY 

   0                                                  1
0     ----------------------------------------------...
1     University College of Engineering Vizianagaram...
2     JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY KAKI...
3     HTNO:18VV1A3105 -- II B.Tech I Sem [R17] Regul...
4                  NO\nSUBCODE\nSUBNAME\nGRADE\nCREDITS
5     1\nR1721011\nPROBABILITY & STATISTICS\nS\n3 ( ...
6     2\nR1721311\nMECHANICS OF SOLIDS AND FLUIDS\nA...
7     3\nR1721312\nMETALLURGICAL THERMO DYNAMICS-I\n...
8                  4\nR1721313\nPOLYMERS\nA\n3 ( Pass )
9     5\nR1721314\nPRINCIPLES OF EXTRACTIVE METALLUR...
10      6\nR1721315\nPHYSICAL METALLURGY\nB\n3 ( Pass )
11    7\nR1721316\nPHYSICAL METALLURGY LABORATORY\nO...
12    8\nR1721317\nMECHANICS OF SOLIDS LABORATORY\nO...
13                                            TOTAL\n22
14    ----------------------------------------------...
15    University College of Engineering Vizianagaram...
16    JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY 

In [5]:
def row(roll_index,index,x):
    roll=re.search("\d\dVV\dA\d\d\d\d",x.df.iloc[roll_index,1].splitlines()[0]).group()
    l=x.df.iloc[index,1].splitlines()
    l.append(roll)
    l[4]=l[4][0]
    y=np.reshape(l,(1,6))
    return [y,roll_index,index+1]
l,b,c=row(3,5,x[0])
a=pd.DataFrame(l,columns=['NO', 'SUBCODE', 'SUBNAME', 'GRADE', 'CREDITS',"roll_no"])

In [6]:
int(l[0][5][0:2])

16

In [8]:
df_regular=pd.DataFrame()
df_lateral=pd.DataFrame()
df_supply=pd.DataFrame()
def f(x, year):
    roll_index=3
    index=roll_index+2
    regular=pd.DataFrame(columns=['NO', 'SUBCODE', 'SUBNAME', 'GRADE', 'CREDITS',"roll_no"])
    supply=pd.DataFrame(columns=['NO', 'SUBCODE', 'SUBNAME', 'GRADE', 'CREDITS',"roll_no"])
    lateral=pd.DataFrame(columns=['NO', 'SUBCODE', 'SUBNAME', 'GRADE', 'CREDITS',"roll_no"])
    while(index<=len(x.df.iloc[:,1])):
        z=x.df.iloc[index,1].splitlines()[0]
        y="TOTAL"
        i=0
        while(z!=y):

            l,roll_index,index=row(roll_index,index,x)
            a=pd.DataFrame(l,columns=['NO', 'SUBCODE', 'SUBNAME', 'GRADE', 'CREDITS',"roll_no"])
            print(l[0][5][4])
            if(l[0][5][4]=="1"):
                if(int(l[0][5][0:2])!=year):
                    supply=supply.append(a)
                else:
                    regular=regular.append(a)
            elif(l[0][5][4]=="5"):
                if(l[0][5][0:2]!=str(year+1)):
                    supply=supply.append(a)
                else:
                    regular=regular.append(a)
     
            i=i+1
            z=x.df.iloc[index,1].splitlines()[0]
        roll_index=index+4
        index=roll_index+2
        #print(df)
    return [regular,supply]
for i in x:
    l=f(i,18)
    df_regular=df_regular.append(l[0])
    df_supply=df_supply.append(l[1])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [9]:
df_regular

,NO,SUBCODE,SUBNAME,GRADE,CREDITS,roll_no
0,1,R1721011,PROBABILITY & STATISTICS,B,3,18VV1A0101
0,2,R1721012,BASIC ELECTRICAL AND ELECTRONICS ENGINEERING,S,3,18VV1A0101
0,3,R1721013,FLUID MECHANICS,A,3,18VV1A0101
0,4,R1721014,STRENGTH OF MATERIALS - I,S,3,18VV1A0101
0,5,R1721015,SURVEYING,S,3,18VV1A0101
...,...,...,...,...,...,...
0,4,R1721313,POLYMERS,D,3,19VV5A3141
0,5,R1721314,PRINCIPLES OF EXTRACTIVE METALLURGY,C,3,19VV5A3141
0,6,R1721315,PHYSICAL METALLURGY,D,3,19VV5A3141
0,7,R1721316,PHYSICAL METALLURGY LABORATORY,S,2,19VV5A3141


In [10]:
x=df_regular

In [11]:
x

,NO,SUBCODE,SUBNAME,GRADE,CREDITS,roll_no
0,1,R1721011,PROBABILITY & STATISTICS,B,3,18VV1A0101
0,2,R1721012,BASIC ELECTRICAL AND ELECTRONICS ENGINEERING,S,3,18VV1A0101
0,3,R1721013,FLUID MECHANICS,A,3,18VV1A0101
0,4,R1721014,STRENGTH OF MATERIALS - I,S,3,18VV1A0101
0,5,R1721015,SURVEYING,S,3,18VV1A0101
...,...,...,...,...,...,...
0,4,R1721313,POLYMERS,D,3,19VV5A3141
0,5,R1721314,PRINCIPLES OF EXTRACTIVE METALLURGY,C,3,19VV5A3141
0,6,R1721315,PHYSICAL METALLURGY,D,3,19VV5A3141
0,7,R1721316,PHYSICAL METALLURGY LABORATORY,S,2,19VV5A3141


In [12]:
def shit(df):
    df.rename(columns={'SUBNAME':'Subname','GRADE':'Grade','CREDITS':'Credits','roll_no':'Htno','NO':'Sno'},inplace=True)
    df.drop('SUBCODE',axis=1,inplace=True)
    df.set_index('Sno',inplace=True)
    df['Credits']=df['Credits'].astype('int')
    #chainging to requried format
    df['points']=df['Grade'].map({'O':10,'S':9,'A':8,'B':7,'C':6,'D':5,'F':0,"ABSENT":0})
    df['Credits']=df['Credits'].apply(lambda x:int(str(x).split(" ")[0]))
    df['Total']=df['points']*df['Credits']
    df.drop(['Grade','points'],axis=1,inplace=True)
    df['sep']=df['Htno'].apply(lambda x:x[6:8])
    #division based on roll numbers
    unique=df['sep'].unique()
    seperate=[]#joining the rows with same Htno
    for i in range(len(unique)):
        seperate.append(df[df.sep==unique[i]])
    return seperate
   

In [13]:
y=shit(x)

In [14]:
y

[                                          Subname  Credits        Htno  Total  \
 Sno                                                                             
 1                        PROBABILITY & STATISTICS        3  18VV1A0101     21   
 2    BASIC ELECTRICAL AND ELECTRONICS ENGINEERING        3  18VV1A0101     27   
 3                                 FLUID MECHANICS        3  18VV1A0101     24   
 4                       STRENGTH OF MATERIALS - I        3  18VV1A0101     27   
 5                                       SURVEYING        3  18VV1A0101     27   
 ..                                            ...      ...         ...    ...   
 4                       STRENGTH OF MATERIALS - I        3  19VV5A0139     27   
 5                                       SURVEYING        3  19VV5A0139     30   
 6                   BUILDING PLANNING AND DRAWING        3  19VV5A0139     27   
 7                          SURVEYING FIELD WORK-I        2  19VV5A0139     20   
 8              

In [18]:
def requriedformat(seperate):
    table=[]
    for i in range(len(seperate)):
        table.append(seperate[i].pivot_table(index='Htno',columns='Subname',values='Total',fill_value=0))
        table[i]['Credits']=seperate[i].groupby('Htno')['Credits'].sum()
    return table


In [20]:
table=requriedformat(y)

In [22]:
def SGPAcalculation(table):
    for i in range(len(table)):
        table[i]['Total']=table[int(i)].iloc[:,:-1].sum(axis=1)
        table[i]['SGPA']=(table[i]['Total']/table[i]['Credits'])
        table[i].reset_index(inplace=True)
        table[i].index = np.arange(1, len(table[i]) + 1)
        table[i]=table[i].rename_axis('SNo',axis=1)
    return table

In [ ]:
table=SGPAcalculation(table)

In [28]:
def Analysis(table):
    an=[]
    pp=[]
    for i in range(len(table)):
        n=list(table[i].columns).index('Htno')
        m=list(table[i].columns).index('Credits')
        analysis={'No of pass subwise':list(table[i][table[i]==0].iloc[:,n+1:m].isnull().sum()),
        'Above 7 gpa':len(table[i][table[i]['SGPA']>=7]),
        'Above 6 gpa':len(table[i][table[i]['SGPA']>=6]),
        'Above 4 gpa':len(table[i][table[i]['SGPA']>=4]),
        'Below 4 gpa':len(table[i][table[i]['SGPA']<4]),
        }
        c=pd.DataFrame(analysis)
        c=c.T
        #c.columns=(table[i].columns)[n+1:m]
        an.append(c)
        pa={'pass':[len(table[i][table[i]['Credits']==22]['Credits'])],
         'pass per':100-int((len(table[i][table[i]['Credits']<22]['Credits'])/len(table[i]))*100)
        }
        d=pd.DataFrame(pa)
        d=d.T
        pp.append(d)
    return pp
        
pp=Analysis(table)

In [29]:
pp

[           0
 pass      25
 pass per  93,
            0
 pass      49
 pass per  84,
            0
 pass      32
 pass per  53,
            0
 pass      58
 pass per  91,
            0
 pass      48
 pass per  75,
            0
 pass      52
 pass per  87,
            0
 pass      16
 pass per  50]

In [ ]:
import matplotlib.pyplot as plt
for i in range(len(table)):
    n=list(table[i].columns).index('Htno')
    m=list(table[i].columns).index('Credits')
    subjects=list(table[i].columns)[n+1:m]
    labels=list(an[i].index)
    for j in range(len(labels)):
        values=an[i][j]
        fig1, ax1 = plt.subplots()
        ax1.pie(values, autopct='%1.1f%%',
        shadow=True, startangle=90)
        ax1.axis('equal')
        plt.legend(labels,loc='best')
        plt.title(subjects[j])
        plt.show()
    break

In [ ]:
#writingtoexcel
writer = pd.ExcelWriter('grades.xlsx', engine='xlsxwriter')
for i in range(len(table)):
    styler_obj[i].to_excel(writer,sheet_name='Sheet'+str(i))
    an[i].to_excel(writer,sheet_name='Sheet'+str(i), startrow=len(table[i])+2, index=True,header=None)
    pp[i].to_excel(writer,sheet_name='Sheet'+str(i), startrow=(len(table[i])+len(an[i]))+5,index=True,header=None)
    
writer.save()

In [ ]:
for i in range(len(table)):
    writer=pd.ExcelWriter("grades"+str(i)+".xlsx",engine="xlsxwriter")
    workbook=writer.book
    styler_obj[i].to_excel(writer,sheet_name="sheet"+str(i))
    an[i].to_excel(writer,sheet_name="sheet"+str(i+1),index=True,header=None)
    worksheet=writer.sheets["sheet"+str(i)]
    chart=workbook.add_chart({"type":"pie"})
    chart.add_series({
        "values":["sheet"+str(i+1),0,i,4,i],
    })
    worksheet.insert_chart("A4",chart)
    writer.save()